In [0]:
!nvidia-smi
############################################################
#  Program Settings
############################################################
specific_model = 'full-pallet'  # 'product-on-pallet' or 'full-pallet'
recent_weights = '/content/drive/My Drive/Colab Notebooks/mrcnn/mask_rcnn_coco.h5' # '/content/drive/My Drive/Colab Notebooks/full-pallet/logs/full-pallet20191210T0145/mask_rcnn_full-pallet_0039.h5'
setting = 'test' 
train_with = 'all' # heads or all
allow_augmentation = True


import sys
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
sys.path.append('/content/drive/My Drive/Colab Notebooks/mcrnn/')
import os
import json
from keras.models import load_model
import numpy as np
import skimage.draw
import matplotlib.pyplot as plt
from mrcnn.config import Config
from mrcnn import visualize
from mrcnn import model as modellib, utils
from mrcnn.visualize import display_images
from mrcnn.model import log
# %tensorflow_version 1.x

############################################################
#  Configurations
############################################################

class ProductsConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = specific_model
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1 # Batch size to train on
    # Number of classes (including background)
    NUM_CLASSES = 1 + 1  # Background + object of interest
    STEPS_PER_EPOCH = len(os.listdir('/content/drive/My Drive/Colab Notebooks/'+specific_model+'/select_images_cnn/train/'))
    print(STEPS_PER_EPOCH)
    DETECTION_MIN_CONFIDENCE = 0.965

    if specific_model != 'product-on-pallet': 
        # Some stuff to speed it up, comment out if necessary
        MAX_GT_INSTANCES = 100
        IMAGE_MIN_DIM = 512
        IMAGE_MAX_DIM = 512
        TRAIN_ROIS_PER_IMAGE = 200
        # BACKBONE = "resnet50"
        # Resnet 50 is quicker but less accurate, might be just fine though


############################################################
#  Dataset
############################################################

class ProductsDataset(utils.Dataset):

    def load_images(self, subset):
        print('subset =', subset)
        assert subset in ['train', 'val', 'test']
        # Add classes. We have only one class to add.
        self.add_class(specific_model, 1, specific_model)

        if (subset == 'train') or (subset == 'val'):
            # Path to the json that contains the metadata
            f = open('/content/drive/My Drive/Colab Notebooks/'+specific_model+'/select_images_cnn/'+subset+'data.json')
            jsondat = json.load(f)

            # All of the annotations for all of the images
            annotations = list(jsondat.values())  # don't need the dict keys

            # The annotate tool saves images in the JSON even if they don't have any
            # annotations. Skip unannotated images.
            annotations = [a for a in annotations if a['regions']]

            # Add images
            n = 1
            for a in annotations:
                # Get the x, y coordinaets of points of the polygons that make up
                # the outline of each object instance. These are stores in the
                # shape_attributes (see json format above)
                # The if condition is needed to support VIA versions 1.x and 2.x.


                # problem with these ones apparently, no idea what - images and annotations look good...
                issues = ['SamsClub6235_110119_080838_872812.jpg', 
                          'SamsClub6235_110119_083325_616724.jpg', 
                          'SamsClub6235_110119_075403_627780.jpg',
                          'SamsClub6235_110119_075522_618224.jpg']

                if a['filename'] in issues:
                  continue

                if type(a['regions']) is dict:
                    polygons = [r['shape_attributes'] for r in a['regions'].values()]
                else:
                    polygons = [r['shape_attributes'] for r in a['regions']]

                # load_mask() needs the image size to convert polygons to masks.
                # Unfortunately, VIA doesn't include it in JSON, so we must read
                # the image. This is only managable since the dataset is tiny.
                image_path = '/content/drive/My Drive/Colab Notebooks/'+specific_model+'/select_images_cnn/'+subset+'/'+a['filename']
                image = skimage.io.imread(image_path)
                print(n)
                n += 1
                height, width = image.shape[:2]
                
                self.add_image(
                    specific_model,
                    image_id=a['filename'],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons)
          
        elif subset == 'test':
            path = '/content/drive/My Drive/Colab Notebooks/'+specific_model+'/select_images_cnn/test/'
            annotations = os.listdir(path)
            print(path)
            print(annotations)
            for a in annotations:
                image_path = path + a
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

                self.add_image(
                    specific_model,
                    image_id=a,  
                    path=image_path,
                    width=width, height=height,
                    polygons=None)
              
    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """

        # If not a balloon dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != specific_model:
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape [height, width, instance_count]
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        return mask.astype(np.bool), np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == specific_model:
            return info["path"]


def train(model, num_epochs):

    # Image Augmentation if allow_augmentation = True
    # Right/Left flip 50% of the time
    augmentation = None
    print(allow_augmentation)
    if allow_augmentation:
      import imgaug.augmenters
      augmentation = imgaug.augmenters.Fliplr(0.5)

    # Training dataset.
    print('Making train set')
    dataset_train = ProductsDataset()
    dataset_train.load_images("train")
    dataset_train.prepare()

    # Validation dataset
    print('Making validation set')
    dataset_val = ProductsDataset()
    dataset_val.load_images("val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=num_epochs,
                augmentation = augmentation,
                layers = train_with) # previously augmentation=None, layers='heads'


############################################################
#  Model stuff
############################################################

if setting == 'train':
    # Path to trained weights file
    WEIGHTS_PATH = recent_weights
    config = ProductsConfig()
    model = modellib.MaskRCNN(mode="training", config=config, model_dir='/content/drive/My Drive/Colab Notebooks/'+specific_model+'/logs')
    # pre-trained weights to start with
    weights_path = WEIGHTS_PATH
    model.load_weights(weights_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"])

    # print('Training...')
    train(model, num_epochs=200)

elif setting == 'test':
    def get_ax(rows=1, cols=1, size=16):
      #  Adjust the size attribute to control how big to render images
      fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
      return fig, ax
    
    config = ProductsConfig()
    model = modellib.MaskRCNN(mode="inference", config=config, model_dir='/content/drive/My Drive/Colab Notebooks/'+specific_model+'/logs')
    weights_path = model.find_last()
    print(weights_path)
    model.load_weights(weights_path, by_name=True)

    path = '/content/drive/My Drive/Colab Notebooks/'+specific_model+'/select_images_cnn/test/'

    for f in os.listdir(path):
        image = skimage.io.imread(path + f)
        results = model.detect([image], verbose=1)

        # # Display results
        fig, ax = get_ax(1)
        r = results[0]
        visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                                    specific_model, r['scores'], ax=ax,
                                    title="Predictions")
        
        # fig.savefig('/content/drive/My Drive/Colab Notebooks/'+specific_model+'/outputs_test/' + f)
        # plt.close()

# elif setting == 'eval':
#     def get_ax(rows=1, cols=1, size=16):
#       #  Adjust the size attribute to control how big to render images
#       fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
#       return fig, ax


#     dataset = ProductsDataset()
#     dataset.load_images(subset="val")
      
#     # Must call before using the dataset
#     dataset.prepare()

#     # Load in the model
#     config = ProductsConfig()
#     model = modellib.MaskRCNN(mode="inference", config=config, model_dir='/content/drive/My Drive/Colab Notebooks/logs')
#     weights_path = model.find_last()
#     model.load_weights(weights_path, by_name=True)

#     for f in dataset.image_ids:
#         image_id = f
#         image, image_meta, gt_class_id, gt_bbox, gt_mask =\
#         modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
#         info = dataset.image_info[image_id]
#         # print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
#         #                                       dataset.image_reference(image_id)))

#         # Run object detection
#         results = model.detect([image], verbose=1)

#         # Display results
#         # fig, ax = plt.subplots()
#         fig, ax = get_ax(1)
#         r = results[0]
#         visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                                     dataset.class_names, r['scores'], ax=ax,
#                                     title="Predictions")

#         # fig.savefig('/content/drive/My Drive/Colab Notebooks/outputs_val/' + info['id'])
#         # plt.close()



Wed Dec 11 20:38:42 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    34W / 250W |    267MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

TypeError: ignored